# <center> Исследование данных hr-агентства

## **Цели и задачи исследования**

*HR*-агентство изучает тренды на рынке труда в *IT*. Компания хочет провести исследование на основе данных о зарплатах в сфере *Data Science* за 2020–2022 годы и получить некоторые выводы.

Необходимо исследовать данные и сделайте выводы по полученным результатам. Подкрепить свои рассуждения и выводы визуализациями. С помощью статистического тестирования проверить, являются ли выводы статистически значимыми.

В процессе своего анализа необходимо:
1. Выяснить, какие факторы влияют на зарплату специалиста Data Scientist.
2. Ответить на ключевые вопросы HR-агентства:
    - Наблюдается ли ежегодный рост зарплат у специалистов Data Scientist?
    - Как соотносятся зарплаты Data Scientist и Data Engineer в 2022 году?
    - Как соотносятся зарплаты специалистов Data Scientist в компаниях различных размеров?
    - Есть ли связь между наличием должностей Data Scientist и Data Engineer и размером компании?

Если будут найдены интересные закономерности, также необходимо отметить в анализе.

Необходимо продемонстрировать использование разных тестов для проверки статистической значимости сделанных выводов:
- тесты для количественного признака:
  - для одной выборки;
  - для двух выборок;
  - для нескольких выборок;
- тест для категориальных признаков.

## **Описание данных**

Исходные данные можно скачать по [ссылке](https://lms-cdn.skillfactory.ru/assets/courseware/v1/9e84f30c5bc84881a5e33262d5e32a8b/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/ds_salaries.zip).

Исходные данные содержат следующие столбцы:

| <center> **Наименование столбца** | <center> **Описание** |
|--|--|
|`work_year`|Год, в котором была выплачена зарплата.|
|`experience_level`|Опыт работы на этой должности в течение года со следующими возможными значениями:<br> • *EN — Entry-level/Junior;*<br> • *MI — Mid-level/Intermediate;*<br> • *SE — Senior-level/Expert;*<br> • *EX — Executive-level/Director.*|
|`employment_type`|Тип трудоустройства для этой роли:<br> • *PT* — неполный рабочий день;<br> • *FT* — полный рабочий день;<br> • *CT* — контракт;<br> • *FL* — фриланс.<br>|
|`job_title`|Роль, в которой соискатель работал в течение года.|
|`salary`|Общая выплаченная валовая сумма заработной платы.|
|`salary_currency`|Валюта выплачиваемой заработной платы в виде кода валюты *ISO* 4217.|
|`salary_in_usd`|Зарплата в долларах США (валютный курс, делённый на среднее значение курса доллара США<br> за соответствующий год через *fxdata.foorilla.com*).|
|`employee_residence`|Основная страна проживания сотрудника в течение рабочего года в виде кода страны *ISO* 3166.|
|`remote_ratio`|Общий объём работы, выполняемой удалённо. Возможные значения:<br> • 0 — удалённой работы нет (менее 20 %);<br> • 50 — частично удалённая работа;<br> • 100 — полностью удалённая работа (более 80 %).|
|`company_location`|Страна главного офиса работодателя или филиала по контракту в виде кода страны *ISO* 3166.|
|`company_size`|Среднее количество людей, работавших в компании в течение года:<br> • *S* — менее 50 сотрудников (небольшая компания);<br> • *M* — от 50 до 250 сотрудников (средняя компания);<br> • *L* — более 250 сотрудников (крупная компания).|

Оригинальный датасет: [Data Science Job Salaries” (kaggle.com)](https://www.kaggle.com/datasets/ruchi798/data-science-job-salaries)


## **1. Загрузка и обработка данных**

In [1]:
# Импортирую библиотеки
import pandas as pd
import numpy as np
import statistics
import category_encoders as ce 
import scipy.stats as stats

# Для графиков
import matplotlib.pyplot as plt
import seaborn as sns

# Для красоты
sns.set_theme('notebook', style="whitegrid", palette="Set2")

In [2]:
# Загружаем данные
data = pd.read_csv('data/ds_salaries.zip', index_col = 0)
# Проверим, что данные загружены корректно
data.head(3)

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M


In [3]:
# Получим основную информацию о данных
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 607 entries, 0 to 606
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   work_year           607 non-null    int64 
 1   experience_level    607 non-null    object
 2   employment_type     607 non-null    object
 3   job_title           607 non-null    object
 4   salary              607 non-null    int64 
 5   salary_currency     607 non-null    object
 6   salary_in_usd       607 non-null    int64 
 7   employee_residence  607 non-null    object
 8   remote_ratio        607 non-null    int64 
 9   company_location    607 non-null    object
 10  company_size        607 non-null    object
dtypes: int64(4), object(7)
memory usage: 56.9+ KB


Данные загружены корректно. В таблице 607 записей, пропущенные значения отсутствуют. 
У нас 4 числовых (целочисленных) признака и 7 текстовых признаков. 

Имеются избыточные признаки 'salary' и 'salary_currency' - заработная плата сотрудников 
уже пересчитана в доллары в признаке 'salary_in_usd'. Для упрощения анализа удалим 
эти признаки, так как работа с данными в единой валюте (долларах) более удобна.

После удаления проверим данные на дубликаты:

### **Очистка данных**

In [4]:
# Удаление неинформативных признаков
columns_to_drop = ['salary', 'salary_currency']
data.drop(columns=columns_to_drop, inplace=True)

# Проверка данных на дубликаты и удаление их при наличии
duplicates_count = data.duplicated().sum()
if duplicates_count > 0:
    print(f'Обнаружено дубликатов: {duplicates_count}')
    data.drop_duplicates(inplace=True)
    print(f'Удалено {duplicates_count} дубликата')
else:
    print('Дубликатов не обнаружено')

Обнаружено дубликатов: 42
Удалено 42 дубликата


### **Определение типа данных**

In [5]:
# Выведем количество уникальных значений и тип данных каждого столбца
def get_unique(data):
    """Возвращает DataFrame с количеством уникальных значений и типами данных."""
    result = []
    for column in data.columns:
        result.append({
            'Столбец': column,
            'Уникальные_значения': data[column].nunique(),
            'Тип_данных': str(data[column].dtype)})
    return pd.DataFrame(result)

get_unique(data)

,Столбец,Уникальные_значения,Тип_данных
0,work_year,3,int64
1,experience_level,4,object
2,employment_type,4,object
3,job_title,50,object
4,salary_in_usd,369,int64
5,employee_residence,57,object
6,remote_ratio,3,int64
7,company_location,50,object
8,company_size,3,object


In [6]:
# В столбце work_year всего 3 значения, хотя он числовой
# Выведем содержимое столбца
data['work_year'].unique().tolist()

[2020, 2021, 2022]

Числовые данные - это количественные данные. Категориальные - качественные. Если признак имеет ограниченное число значений и математические операции над ним не имеют смысла - он категориальный. Для ускорения работы, преобразуем категориальные признаки с числом уникальных значений < 15 в категориальный тип данных.

|**Признак**|<center>**Тип**|**Итоговый тип данных**|
|--|--|--|
|`work_year`|категориальный (2020, 2021, 2022)|`category`|
|`experience_level`|категориальный (EN, MI, SE, EX)|`category`|
|`employment_type`|категориальный (PT, FT, CT, FL)|`category`|
|`job_title`|категориальный|`object`|
|`salary_in_usd`|числовой|`int64`|
|`employee_residence`|категориальный|`object`|
|`remote_ratio`|категориальный (0, 50, 100)|`category`|
|`company_location`|категориальный|`object`|
|`company_size`|категориальный (S, M, L)|`category`|

In [7]:
# Преобразование категориальных признаков с малым числом уникальных значений

columns_to_category = [
    'work_year',           # год (3 уникальных значения)
    'experience_level',    # уровень опыта (4 значения) 
    'employment_type',     # тип занятости (4 значения)
    'remote_ratio',        # режим работы (3 значения)
    'company_size',        # размер компании (3 значения)
]
data[columns_to_category] = data[columns_to_category].astype('category')

### **Основные статистические характеристики**

In [8]:
# Основные статистические характеристики числовых данных
data.describe(include='int')

,salary_in_usd
count,565.000000
mean,110610.343363
std,72280.702792
min,2859.000000
25%,60757.000000
50%,100000.000000
75%,150000.000000
max,600000.000000


In [9]:
# Основные статистические характеристики категориальных данных
data.describe(exclude='int')

,work_year,experience_level,employment_type,job_title,employee_residence,remote_ratio,company_location,company_size
count,565,565,565,565,565,565,565,565
unique,3,4,4,50,57,3,50,3
top,2022,SE,FT,Data Scientist,US,100,US,M
freq,278,243,546,130,295,346,318,290


**Выводы:**

- Для числового признака (заработная плата):
  - Средняя зарплата: 110 610 долларов;
  - Большой разброс зарплат: от 2 859 до 600 000 долларов;
  - Стандартное отклонение: 72 280 долларов.

- Для категориальных признаков:
  - Самое большое количество записей за 2022 год (278);
  - Преобладающий опыт работы: SE (Senior);
  - Самый частый тип занятости: FT (Full-time);
  - Популярная должность: Data Scientist;
  - Местоположение: US лидирует по расположению как компаний, так и сотрудников;
  - Режим работы: полностью удаленная работа лидирует;
  - Размер компании: в большей части записей компании среднего размера (М).

## **2. Разведывательный анализ данных**

### **2.1. Визуальный анализ данных**

#### **Анализ числовых признаков**

В данных остался только один числовой признак - заработная плата. Построим для него гистограмму и коробчатую диаграмму:

In [10]:
# Построим графики распределения заработной платы
fig, axes = plt.subplots(2, 1, figsize=(10, 10))

salary_data = data['salary_in_usd']
mean_salary = salary_data.mean()

sns.boxplot(
    data, 
    x='salary_in_usd', 
    ax=axes[0], 
    orient='h', 
    medianprops={'color': 'red', 'linestyle': '--'}, 
    legend=False,
)
axes[0].set_title('Распределение зарплат в USD')
axes[0].set(xlabel='Размер зарплаты')

sns.histplot(data, kde=True, x='salary_in_usd', bins=30, ax=axes[1])
axes[1].set(xlabel='Размер зарплаты', ylabel='Частота')
axes[1].axvline(
    mean_salary, 
    color='red', 
    linestyle='--', 
    label=f'Среднее: ${mean_salary:.0f}',
)

# Для отображения раскомментировать plt.show()
# plt.show()
plt.close()

# Выведем меры центральной тенденции
print(f"• Медиана: {statistics.median(salary_data):.0f}$")
print(f"• Мода: {statistics.mode(salary_data):.0f}$")
print(f"• Среднее: {mean_salary:.0f}$")
print(f"• 75% данных ниже: {salary_data.quantile(0.75):.0f}$")

• Медиана: 100000$
• Мода: 100000$
• Среднее: 110610$
• 75% данных ниже: 150000$


<div align="center">
<image src="data/image/plt_1.png"></div>

**Вывод.** Распределение зарплат существенно отклоняется от нормального, имеет две "вершины" и правую асимметрию. Большинство значений зарплаты до 150 000 долларов, мода и медиана - 100 000 долларов. При этом среднее значение составило 110 000 долларов, что обусловлено большим разбросом (заработная плата доходит до 600 000 долларов). 600 000 в данном случае похоже на выброс, но возможно такая большая зарплата связана с высокой квалификацией сотрудника.

#### **Анализ категориальных признаков**

Для каждой категории исследуем количество записей и влияние на заработную плату. В процессе анализа возможно преобразуем некоторые категории, если это будет необходимо. План построить следующие графики для анализа данных и ответа на вопросы:

1. work_year - количество записей по годам, имеется ли ежегодный рост зарплат, наблюдается ли ежегодный рост зарплат у специалистов DS и DE, как соотносятся зарплаты Data Scientist и Data Engineer в 2022 году;
2. experience_level - распределение по уровням опыта, влияние опыта на зарплату;
3. employment_type - распределение по графику работы, влияние графика работы на зарплату;
4. job_title - распределение по типу должности, уровень зарплаты в зависимости от должности;
5. remote_ratio - распределение по типу работы, влияние типа работы на заработную плату;
6. company_location, employee_residence - распределение данных по местоположению, коррелируют ли между собой данные признаки;
8. company_size - распределение по размеру компании, уровень зарплаты в зависимости от размера компании, соотносятся ли зарплаты специалистов Data Scientist в компаниях различных размеров, есть ли связь между наличием должностей Data Scientist и Data Engineer и размером компании.

**1. Анализ признака - work_year (год работы).**

In [11]:
# Построим четыре графика
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Количество записей в разрезе года
axes[0][0] = sns.countplot(
    data,
    x='work_year',
    hue='work_year',
    ax=axes[0][0],
    legend=False
)
axes[0][0].set(xlabel='Год', ylabel='Количество записей')
axes[0][0].set_title('Распределение количество записей по годам', fontweight='bold')

# Имеется ли ежегодный рост заработной платы по всем должностям?
axes[0][1] = sns.boxplot(
    data,
    x='work_year',
    hue='work_year',
    y='salary_in_usd',
    ax=axes[0][1],
    legend=False
)
axes[0][1].set(xlabel='Год', ylabel='Уровень заработной платы, USD')
axes[0][1].set_title('Распределение зарплаты по годам для всех должностей', fontweight='bold')

# Имеется ли ежегодный рост заработной платы DS?
ds_mask = data['job_title'] == 'Data Scientist'
axes[1][0] = sns.boxplot(
    data[ds_mask],
    x='work_year',
    hue='work_year',
    y='salary_in_usd',
    ax=axes[1][0],
    legend=False
)
axes[1][0].set(xlabel='Год', ylabel='Уровень заработной платы, USD')
axes[1][0].set_title('Распределение зарплаты по годам для Data Scientist', fontweight='bold')

# Имеется ли ежегодный рост заработной платы DE?
de_mask = data['job_title'] == 'Data Engineer'
axes[1][1] = sns.boxplot(
    data[de_mask],
    x='work_year',
    hue='work_year',
    y='salary_in_usd',
    ax=axes[1][1],
    legend=False
)
axes[1][1].set(xlabel='Год', ylabel='Уровень заработной платы, USD')
axes[1][1].set_title('Распределение зарплаты по годам для Data Engineer', fontweight='bold')

# Для отображения раскомментировать plt.show()
# plt.show()
plt.close()

# Выведем отдельно статистику по данным
stat_year_for_all = (
    data.groupby('work_year', observed=False)['salary_in_usd']
    .describe()
    .round()
)
print('Статистика уровня заработной платы по годам по всем должностям')
display(stat_year_for_all)

stat_year_for_ds = (
    data[ds_mask]
    .groupby('work_year', observed=False)['salary_in_usd']
    .describe()
    .round()
)
print('Статистика уровня заработной платы по годам для Data Scientist')
display(stat_year_for_ds)

stat_year_for_de = (
    data[de_mask]
    .groupby('work_year', observed=False)['salary_in_usd']
    .describe()
    .round()
)
print('Статистика уровня заработной платы по годам для Data Engineer')
display(stat_year_for_de)

Статистика уровня заработной платы по годам по всем должностям


,count,mean,std,min,25%,50%,75%,max
work_year,,,,,,,,
2020,72.0,95813.0,82832.0,5707.0,45724.0,75544.0,115526.0,450000.0
2021,215.0,99430.0,80304.0,2859.0,50000.0,82528.0,135000.0,600000.0
2022,278.0,123089.0,59889.0,10000.0,78791.0,120000.0,160000.0,405000.0


Статистика уровня заработной платы по годам для Data Scientist


,count,mean,std,min,25%,50%,75%,max
work_year,,,,,,,,
2020,21.0,85971.0,81437.0,21669.0,42197.0,62726.0,105000.0,412000.0
2021,44.0,70216.0,45844.0,2859.0,32794.0,67234.0,100923.0,165000.0
2022,65.0,131367.0,55785.0,18442.0,95550.0,135000.0,170000.0,260000.0


Статистика уровня заработной платы по годам для Data Engineer


,count,mean,std,min,25%,50%,75%,max
work_year,,,,,,,,
2020,11.0,88162.0,46473.0,33511.0,51320.0,74130.0,111436.0,188000.0
2021,31.0,79435.0,49246.0,4000.0,29452.0,76833.0,110888.0,200000.0
2022,79.0,124652.0,57847.0,25000.0,78526.0,115000.0,160040.0,324000.0


<div align="center">
<image src="data/image/plt_2.png"></div>

**Вывод:** Количество записей ежегодно растет, с максимальным охватом в 2022 году. Наблюдается значительный рост медианных зарплат в 2022 году для всех категорий специалистов. Data Scientist и Data Engineer показывают схожую динамику - падение средней зарплаты между 2020-2021 годом и резкий скачок в 2022 году. В 2022 году медианная зарплата Data Scientist (135 000) превышает зарплату Data Engineer (115 000), что указывает на более высокую оплату труда Data Scientist в текущем периоде.

**2. Анализ признака - experience_level (опыт работы)**

In [12]:
# Получаем порядок сортировки по возрастанию количества записей
order = data['experience_level'].value_counts(ascending=True).index

# Строим графики
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

axes[0] = sns.countplot(
    data, 
    x='experience_level', 
    hue='experience_level', 
    ax=axes[0], 
    order=order, 
    legend=False
)
axes[0].set(xlabel='Уровень квалификации', ylabel='Количество записей')
axes[0].set_title('Распределение по опыту работы', fontweight='bold')

axes[1] = sns.boxplot(
    data, 
    x='experience_level', 
    y='salary_in_usd', 
    hue='experience_level', 
    ax=axes[1], 
    order=order, 
    legend=False
)
axes[1].set(xlabel='Уровень квалификации', ylabel='Заработная плата, USD')
axes[1].set_title(
    'Распределение заработной платы в зависимости от опыта', 
    fontweight='bold'
)

# Для отображения раскомментировать plt.show()
# plt.show()
plt.close()

# Статистика по данным
stat_experience = (
    data.groupby('experience_level', observed=False)['salary_in_usd']
    .describe()
    .round()
)
print('Статистика уровня заработной платы по уровням квалификации')
display(stat_experience)

Статистика уровня заработной платы по уровням квалификации


,count,mean,std,min,25%,50%,75%,max
experience_level,,,,,,,,
EN,88.0,61643.0,44396.0,4000.0,27505.0,56500.0,85426.0,250000.0
EX,26.0,199392.0,117071.0,69741.0,130006.0,171438.0,233750.0,600000.0
MI,208.0,87793.0,64119.0,2859.0,47164.0,76940.0,112075.0,450000.0
SE,243.0,138375.0,59956.0,18907.0,99532.0,135000.0,171881.0,412000.0


<div align="center">
<image src="data/image/plt_3.png"></div>

**Вывод:** Анализ подтверждает ожидаемую зависимость: опыт напрямую определяет уровень дохода. EX - редкие (наименьшее количество записей) руководящие позиции, показывают самый высокий уровень заработной платы. 600 000 долларов, которые ранее казались выбросом, вполне могут быть заработной платой отдельно взятого директора. Наибольшее количество записей у MI (middle) и SE (senior) уровней квалификации. Имеется выраженный рост средней заработной платы в зависимости от квалификации.

**3. Анализ признака - employment_type (тип трудоустройства).**

In [13]:
# Получаем порядок сортировки по убыванию количества записей
order = data['employment_type'].value_counts().index

# Строим графики
fig, axes = plt.subplots(1, 2, figsize=(15, 8))

axes[0] = sns.countplot(
    data, 
    x='employment_type', 
    hue='employment_type', 
    ax=axes[0], 
    order=order, 
    legend=False
)
axes[0].set(xlabel='Тип трудоустройства', ylabel='Количество записей')
axes[0].set_title(
    'Распределение по типу трудоустройства', 
    fontweight='bold'
)

axes[1] = sns.boxplot(
    data, 
    x='employment_type', 
    hue='employment_type', 
    y='salary_in_usd', 
    ax=axes[1], 
    order=order, 
    legend=False
)
axes[1].set(
    xlabel='Тип трудоустройства', 
    ylabel='Заработная плата, USD'
)
axes[1].set_title(
    'Распределение з/п в зависимости от типа трудоустройства', 
    fontweight='bold'
)

# Для отображения раскомментировать plt.show()
# plt.show()
plt.close()

# Статистика по данным
stat_employment = (
    data.groupby('employment_type', observed=False)['salary_in_usd']
    .describe()
    .round()
)
print('Статистика уровня заработной платы по типу трудоустройства')
display(stat_employment)

Статистика уровня заработной платы по типу трудоустройства


,count,mean,std,min,25%,50%,75%,max
employment_type,,,,,,,,
CT,5.0,184575.0,156251.0,31875.0,100000.0,105000.0,270000.0,416000.0
FL,4.0,48000.0,40530.0,12000.0,18000.0,40000.0,70000.0,100000.0
FT,546.0,111812.0,70791.0,2859.0,62726.0,100000.0,150000.0,600000.0
PT,10.0,33070.0,31473.0,5409.0,12000.0,18818.0,48370.0,100000.0


In [14]:
# Для вывода
ft_mask = data['employment_type'] == 'FT'
len_FT = round(data[ft_mask].shape[0] / data.shape[0] * 100)
print(f'Преобладающее количество записей ({len_FT}%) приходится на полный рабочий день')

Преобладающее количество записей (97%) приходится на полный рабочий день


<div align="center">
<image src="data/image/plt_4.png"></div>

**Вывод:** Анализ показывает абсолютное доминирование полной занятости (FT), составляющей 97% всех записей. Остальные типы занятости (FL, CT, PT) представлены единичными случаями, что делает статистически незначимыми любые выводы об уровне заработной платы в этих категориях. Наблюдаемые различия в заработной плате для альтернативных форматов занятости могут быть следствием малой выборки и не отражают реальные рыночные тенденции. Таким образом, объективный анализ возможен только для сотрудников на полной занятости.

**4. Анализ признака - job_title (должность).**

In [15]:
# Получаем порядок сортировки по убыванию количества
order = data['job_title'].value_counts().index

plt.figure(figsize=(15, 10))
sns.countplot(data=data, y='job_title', hue='job_title', order=order, legend=False)
plt.title('Распределение должностей', fontweight='bold')
plt.xlabel('Количество вакансий', fontweight='bold')
plt.ylabel('Должность', fontweight='bold')
plt.tight_layout()

# Для отображения раскомментировать plt.show()
# plt.show()
plt.close()

<div align="center">
<image src="data/image/plt_5.png"></div>

**Вывод:** Из графика очевидно, что топ-5 должностей - Data Scientist, Data Engineer, Data Analyst, Maschine Learning Engineer, Research Scientist. В виду того, что рассматриваемые профессии (Data Scientist, Data Engineer) вошли в топ-5 по количеству записей, для целей дальнейшего исследования мы можем переопределить группы на топ-5 профессий и "other"

In [16]:
# Чтобы не менять исходные данные создадим новый DF
df_copy = data.copy()

# Переопределяем данные на топ-5 и other
top_jobs = df_copy['job_title'].value_counts().head(5).index
df_copy['job_title'] = df_copy['job_title'].apply(
    lambda x: x if x in top_jobs else 'Other'
)

# Строим график зависимости заработной платы от должности
order = df_copy['job_title'].value_counts().index
plt.figure(figsize=(8, 5))
sns.boxplot(
    df_copy,
    x='salary_in_usd',
    y='job_title',
    hue='job_title',
    order=order,
    medianprops={'color': 'red', 'linestyle': '--'},
)
plt.title('Распределение заработной платы по должностям', fontweight='bold')
plt.xlabel('Заработная плата, USD', fontweight='bold')
plt.ylabel('Должность', fontweight='bold')

# Для отображения раскомментировать plt.show()
# plt.show()
plt.close()

# Статистика по данным
stat_job_title = (
    df_copy.groupby('job_title', observed=False)['salary_in_usd']
    .describe()
    .round()
    .sort_values('count', ascending=False)
)
print('Статистика уровня заработной платы по типу профессии')
display(stat_job_title)

Статистика уровня заработной платы по типу профессии


,count,mean,std,min,25%,50%,75%,max
job_title,,,,,,,,
Other,177.0,128273.0,91817.0,5409.0,61896.0,114047.0,170000.0,600000.0
Data Scientist,130.0,103336.0,63968.0,2859.0,50330.0,100000.0,140400.0,412000.0
Data Engineer,121.0,109750.0,58204.0,4000.0,66022.0,100800.0,150000.0,324000.0
Data Analyst,82.0,90090.0,40686.0,6072.0,60325.0,90000.0,116112.0,200000.0
Machine Learning Engineer,39.0,101165.0,62519.0,20000.0,49173.0,87425.0,131500.0,250000.0
Research Scientist,16.0,109020.0,98543.0,42000.0,62176.0,76264.0,105000.0,450000.0


<div align="center">
<image src="data/image/plt_6.png"></div>

**Вывод:** Вывод: Анализ распределения зарплат показывает, что категория "Other" демонстрирует наибольший разброс значений и максимальные зарплаты до 600 000 долларов, ввиду того что в данной группе оказались руководящие позиции. Data Scientist (DS) и Data Engineer (DE) имеют сопоставимые медианные значения около 100 000 долларов, при этом DE показывают незначительно более высокую среднюю зарплату, хоть максимальное значение у DS выше. Data Analysts получают наименьшее вознаграждение среди основных групп.

**5. Анализ признака - remote_ratio (удаленность работы).**

In [17]:
# Строим графики
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

axes[0] = sns.countplot(
    data, 
    x='remote_ratio', 
    hue='remote_ratio', 
    ax=axes[0],
    legend=False
)
axes[0].set(
    xlabel='Объем работы выполняемой удаленно', 
    ylabel='Количество записей'
)
axes[0].set_title(
    'Распределение по объему работы выполняемой удаленно', 
    fontweight='bold'
)

axes[1] = sns.boxplot(
    data, 
    x='remote_ratio', 
    hue='remote_ratio', 
    y='salary_in_usd', 
    ax=axes[1],
    legend=False
)
axes[1].set(
    xlabel='Объем работы выполняемой удаленно', 
    ylabel='Заработная плата, USD'
)
axes[1].set_title(
    'Распределение з/п в зависимости от объема работы выполняемой удаленно', 
    fontweight='bold'
)

# Для отображения раскомментировать plt.show()
# plt.show()
plt.close()

# Статистика по данным
stat_remote_ratio = (
    data.groupby('remote_ratio', observed=False)['salary_in_usd']
    .describe()
    .round()
)
print('Статистика уровня заработной платы объему работы выполняемой удаленно')
display(stat_remote_ratio)

Статистика уровня заработной платы объему работы выполняемой удаленно


,count,mean,std,min,25%,50%,75%,max
remote_ratio,,,,,,,,
0,121.0,105785.0,68393.0,2859.0,62000.0,98158.0,136000.0,450000.0
50,98.0,80722.0,57639.0,5409.0,50000.0,68010.0,99926.0,423000.0
100,346.0,120763.0,74930.0,4000.0,70000.0,110712.0,159750.0,600000.0


<div align="center">
<image src="data/image/plt_7.png"></div>

**Вывод:** Сотрудники с полностью удалённым форматом работы составляют большинство в выборке и имеют наибольшую медианную заработную плату. Частичная удалёнка ассоциируется с наименьшими медианными доходами.

**6. Анализ признаков - company_location, employee_residence (местоположение компании и сотрудника).**

После анализа данных нам известно, что данные признаки содержат > 50 каждый. Поэтому для графиков определим ТОП-10 стран, остальные записи обернем в "other".

In [18]:
# Переопределяем данные на топ-10 и other
# Для компаний
top_com_loc = df_copy['company_location'].value_counts().head(10).index
df_copy['company_location'] = df_copy['company_location'].apply(
    lambda x: x if x in top_com_loc else 'Other')

# Для сотрудников
top_res_loc = df_copy['employee_residence'].value_counts().head(10).index
df_copy['employee_residence'] = df_copy['employee_residence'].apply(
    lambda x: x if x in top_res_loc else 'Other')

# Строим графики
fig, axes = plt.subplots(1, 2, figsize=(15, 15))

# Левый график - company_location
company_data = df_copy['company_location'].value_counts()
axes[0].pie(
    company_data.values,
    labels=company_data.index,
    autopct='%1.1f%%',
    startangle=90,
    pctdistance=0.9,
    textprops={'fontsize': 9},
    labeldistance=1.05
)
axes[0].set_title(
    'Страны расположения компаний (ТОП-10)',
    fontweight='bold'
)

# Правый график - employee_residence
residence_data = df_copy['employee_residence'].value_counts()
axes[1].pie(
    residence_data.values,
    labels=residence_data.index,
    autopct='%1.1f%%',
    startangle=90,
    pctdistance=0.9,
    textprops={'fontsize': 9},
    labeldistance=1.05
)
axes[1].set_title(
    'Страны проживания сотрудников (ТОП-10)',
    fontweight='bold'
)

plt.tight_layout()
# Для отображения раскомментировать plt.show()
# plt.show()
plt.close()

# Статистика по данным
stat_com_loc = (
    df_copy.groupby('company_location', observed=False)['salary_in_usd']
    .describe()
    .round()
)
print('Статистика уровня заработной платы в зависимости от местоположения компании')
display(stat_com_loc)

stat_res_loc = (
    df_copy.groupby('employee_residence', observed=False)['salary_in_usd']
    .describe()
    .round()
)
print('Статистика уровня заработной платы в зависимости от местоположения сотрудника')
display(stat_res_loc)

Статистика уровня заработной платы в зависимости от местоположения компании


,count,mean,std,min,25%,50%,75%,max
company_location,,,,,,,,
CA,28.0,100122.0,47081.0,52000.0,70518.0,81896.0,117375.0,225000.0
DE,27.0,81560.0,39903.0,15966.0,58669.0,76833.0,89694.0,173762.0
ES,14.0,53060.0,21091.0,10354.0,40074.0,48372.0,68793.0,87932.0
FR,15.0,63971.0,28680.0,36643.0,48202.0,56738.0,69143.0,152000.0
GB,46.0,81650.0,29847.0,37300.0,56916.0,78526.0,104316.0,183228.0
GR,10.0,52027.0,20357.0,20000.0,41133.0,48680.0,63201.0,87932.0
IN,24.0,28582.0,22698.0,5409.0,16735.0,22124.0,32163.0,94665.0
JP,6.0,114127.0,83585.0,41689.0,66283.0,75682.0,145341.0,260000.0
NL,4.0,54946.0,13381.0,42000.0,44543.0,54021.0,64424.0,69741.0


Статистика уровня заработной платы в зависимости от местоположения сотрудника


,count,mean,std,min,25%,50%,75%,max
employee_residence,,,,,,,,
CA,27.0,97192.0,40152.0,52000.0,71349.0,85000.0,110398.0,196979.0
DE,24.0,85337.0,40216.0,15966.0,62411.0,78015.0,94860.0,173762.0
ES,15.0,57593.0,28395.0,10354.0,41371.0,49461.0,71444.0,130800.0
FR,18.0,59887.0,15849.0,36643.0,49734.0,57920.0,69342.0,93427.0
GB,43.0,81470.0,30284.0,37300.0,55247.0,78526.0,103931.0,183228.0
GR,12.0,56446.0,21545.0,20000.0,43022.0,52209.0,70556.0,88654.0
IN,30.0,37322.0,43741.0,5409.0,16397.0,22124.0,38813.0,200000.0
JP,7.0,103538.0,81283.0,40000.0,52700.0,74000.0,122682.0,260000.0
Other,88.0,60461.0,45688.0,2859.0,25355.0,49000.0,85426.0,230000.0


<div align="center">
<image src="data/image/plt_8.png"></div>

**Вывод**. В обоих распределениях доминируют США, составляя более половины всех записей. Зарплаты в США значительно превышают средние показатели по другим странам. 
На первый взгляд признаки коррелируют между собой. Чтобы проверить это закодируем признак порядковым способом.

In [19]:
# Создаем объект OrdinalEncoder сразу для двух признаков
# Необходимо чтобы коды в них совпадали
cols = ['company_location', 'employee_residence']
ord_encoder = ce.OrdinalEncoder(cols=cols)
df_encoded = ord_encoder.fit_transform(df_copy[cols])
df_copy[cols] = df_encoded[cols]

# Построим Scatter plot
plt.figure(figsize=(8, 5))
plt.scatter(x=df_copy['employee_residence'], y=df_copy['company_location'])
plt.xlabel('Страна проживания сотрудника')
plt.ylabel('Страна расположения компании')
plt.title(
    'Корреляция стран расположения компании и проживания сотрудника',
    fontweight='bold')
plt.tight_layout()
# Для отображения раскомментировать plt.show()
# plt.show()
plt.close()

# Проверим корреляцию (признак категориальный поэтому по Спирмену и Кендаллу)
corr_spearman = df_copy['employee_residence'].corr(df_copy['company_location'], method='spearman')
corr_kendall = df_copy['employee_residence'].corr(df_copy['company_location'], method='kendall')

print(f"Корреляция Спирмена: {corr_spearman:.3f}")
print(f"Корреляция Кендалла: {corr_kendall:.3f}")

Корреляция Спирмена: 0.850
Корреляция Кендалла: 0.823


<div align="center">
<image src="data/image/plt_9.png"></div>

**Вывод**. Наблюдается сильная положительная корреляция (0.826 по Спирмену) между страной проживания сотрудника и расположением компании. Это означает, что в большинстве случаев сотрудники работают в компаниях, расположенных в той же стране, где они проживают.

Высокая корреляция указывает на проблему мультиколлинеарности - признаки employee_residence и company_location содержат избыточную информацию. Возможно необходимо удалить один из признаков, например место проживания сотрудников.

**7. Анализ признака - company_size (размер компании).**

In [20]:
# Построим четыре графика
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Распределение по размеру компании
axes[0, 0] = sns.countplot(
    data=data,
    x='company_size',
    hue='company_size',
    ax=axes[0, 0],
    legend=False
)
axes[0, 0].set(xlabel='Размер компании', ylabel='Количество записей')
axes[0, 0].set_title(
    'Распределение записей по размеру компании', 
    fontweight='bold'
)

# Уровень зарплаты в зависимости от размера компании
axes[0, 1] = sns.boxplot(
    data=data,
    x='company_size',
    y='salary_in_usd',
    hue='company_size',
    ax=axes[0, 1],
    legend=False
)
axes[0, 1].set(xlabel='Размер компании', ylabel='Заработная плата, USD')
axes[0, 1].set_title(
    'Уровень зарплаты в зависимости от размера компании', 
    fontweight='bold'
)

# Зарплаты Data Scientist в компаниях различных размеров
ds_data = data[data['job_title'] == 'Data Scientist']
axes[1, 0] = sns.boxplot(
    data=ds_data,
    x='company_size',
    y='salary_in_usd', 
    hue='company_size',
    ax=axes[1, 0],
    legend=False
)
axes[1, 0].set(xlabel='Размер компании', ylabel='Заработная плата, USD')
axes[1, 0].set_title(
    'Зарплаты Data Scientist по размеру компании', 
    fontweight='bold'
)

# Наличие должностей DS и DE в зависимости от размера компании
de_ds_data = data[data['job_title'].isin(['Data Scientist', 'Data Engineer'])]
axes[1, 1] = sns.countplot(
    data=de_ds_data,
    x='company_size',
    hue='job_title',
    ax=axes[1, 1]
)
axes[1, 1].set(xlabel='Размер компании', ylabel='Количество записей')
axes[1, 1].set_title(
    'Распределение DS и DE по размеру компании', 
    fontweight='bold'
)
axes[1, 1].legend()
# Для отображения раскомментировать plt.show()
# plt.show()
plt.close()

# Статистика
print("Статистика зарплат по размеру компании:")
display(data.groupby('company_size', observed=False)['salary_in_usd'].describe())

print("Статистика зарплат Data Scientist по размеру компании:")
display(ds_data.groupby('company_size', observed=False)['salary_in_usd'].describe())

Статистика зарплат по размеру компании:


,count,mean,std,min,25%,50%,75%,max
company_size,,,,,,,,
L,193.0,118213.880829,86753.270985,5882.0,60000.0,100000.0,153667.00,600000.0
M,290.0,114807.079310,60779.043553,4000.0,70821.5,109640.0,150213.75,450000.0
S,82.0,77872.097561,63814.516062,2859.0,41816.0,65511.0,100000.00,416000.0


Статистика зарплат Data Scientist по размеру компании:


,count,mean,std,min,25%,50%,75%,max
company_size,,,,,,,,
L,43.0,102743.418605,73254.096950,13400.0,45240.5,90734.0,135000.0,412000.0
M,66.0,120080.575758,56347.786862,4000.0,80000.0,119529.5,157500.0,260000.0
S,21.0,51925.761905,33216.289638,2859.0,21669.0,49268.0,82500.0,105000.0


<div align="center">
<image src="data/image/plt_10.png"></div>

**Вывод**. Крупные и средние компании составляют большинство в выборке. Наблюдается зависимость уровня зарплат от размера компании - сотрудники в крупных и средних компаниях получают выше, чем в малых. Различие между средними и крупными компаниями не значительно. Data Scientist в средних компаниях имеют наибольшую медианную зарплату, тогда как в малых и крупных компаниях их доходы ниже. Обе должности (Data Scientist и Data Engineer) наиболее востребованы в средних и крупных компаниях.Наблюдается очевидная зависимость между размером компании и наличием должностей Data Scientist и Data Engineer. 

#### **Корреляция признаков**

Проверим корреляцию между признаками заработной платы, уровня опыта, типа трудоустройства, удаленностью работы и размером компании. Для выявления взаимосвязей в таблице построим тепловую карту корреляции. Предварительно данные необходимо "закодировать". Для кодирования используем OrdinalEncoder. Столбец employment_type переопределим на FT и other (ввиду того что 97% данных FT, остальные данные на мой взгляд не значимы, их можно объединить). Остальные (местоположения, наименование должности и год) не будем включать в матрицу.

In [21]:
# Создаем копию исходных данных для кодирования
df_encoded = data.copy()

# Переопределяем данные для employment_type
df_encoded['employment_type'] = df_encoded['employment_type'].apply(
    lambda x: x if x == 'FT' else 'Other')

# кодируем признаки с OrdinalEncoder
ord_encoder = [
    'experience_level', 'company_size',
    'remote_ratio', 'employment_type'
]
for elem in ord_encoder:
    # Создаем маппинг для кодирования
    if elem == 'experience_level':
        mapping = [{'col': elem, 'mapping': {'EN': 0, 'MI': 1, 'SE': 2, 'EX': 3}}]
    elif elem == 'company_size':
        mapping = [{'col': elem, 'mapping': {'S': 0, 'M': 1, 'L': 2}}]
    elif elem == 'remote_ratio':
        mapping = [{'col': elem, 'mapping': {0: 0, 50: 1, 100: 2}}]
    elif elem == 'employment_type':
        mapping = [{'col': elem, 'mapping': {'Other': 0, 'FT': 1}}]
    encoder = ce.OrdinalEncoder(mapping=mapping)
    df_encoded[[elem]] = encoder.fit_transform(df_encoded[[elem]])

# Удалим лишние признаки
list_for_drop = [
    'job_title', 'employee_residence',
    'company_location', 'work_year'
]
df_encoded.drop(list_for_drop, axis=1, inplace=True)

# Создаем словарь с понятными названиями для признаков
rename_dict = {
    'salary_in_usd': 'Зарплата (USD)',
    'experience_level': 'Уровень опыта',
    'employment_type': 'Тип трудоустройства', 
    'remote_ratio': 'Удаленная работа',
    'company_size': 'Размер компании'
}
# Переименовываем в таблице
df_encoded = df_encoded.rename(columns=rename_dict)

# Рассчитаем корреляцию по Спирману (так как признаки категориальные кроме з/п)
correlation_matrix = df_encoded.corr(method='spearman')

# Строим тепловую карту корреляций
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, 
            annot=True,
            fmt='.2f',
            cmap='RdBu_r')

plt.title('Матрица корреляций между признаками', fontweight='bold')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()

# Для отображения раскомментировать plt.show()
# plt.show()
plt.close()

<div align="center">
<image src="data/image/plt_11.png"></div>

**Вывод**. Наибольшее влияние на заработную плату оказывает уровень опыта (корреляция 0.55), что подтверждает ожидаемую зависимость: с ростом квалификации растет доход. Остальные факторы - размер компании, тип трудоустройства и удаленная работа - демонстрируют слабые связи с зарплатой (корреляции < 0.3), что указывает на их второстепенную роль в формировании дохода.

Данные выводы справедливы для всех должностей в списке, попробуем составить аналогичную матрицу, но только для DS вакансий:

In [22]:
# Оставляем только ds вакансии 
# в переменной ds_mask ранее уже заданы DS должности
df_ds_encoded = data.copy()
df_ds_encoded = df_ds_encoded[ds_mask]

# Переопределяем данные для employment_type
df_ds_encoded['employment_type'] = df_ds_encoded['employment_type'].apply(
    lambda x: x if x == 'FT' else 'Other')

# кодируем признаки с OrdinalEncoder
ord_encoder = [
    'experience_level', 'company_size',
    'remote_ratio', 'employment_type'
]
for elem in ord_encoder:
    # Создаем маппинг для кодирования
    if elem == 'experience_level':
        mapping = [{'col': elem, 'mapping': {'EN': 0, 'MI': 1, 'SE': 2, 'EX': 3}}]
    elif elem == 'company_size':
        mapping = [{'col': elem, 'mapping': {'S': 0, 'M': 1, 'L': 2}}]
    elif elem == 'remote_ratio':
        mapping = [{'col': elem, 'mapping': {0: 0, 50: 1, 100: 2}}]
    elif elem == 'employment_type':
        mapping = [{'col': elem, 'mapping': {'Other': 0, 'FT': 1}}]
    encoder = ce.OrdinalEncoder(mapping=mapping)
    df_ds_encoded[[elem]] = encoder.fit_transform(df_ds_encoded[[elem]])

# Удалим лишние признаки
list_for_drop = [
    'job_title', 'employee_residence',
    'company_location', 'work_year'
]
df_ds_encoded.drop(list_for_drop, axis=1, inplace=True)

# Создаем словарь с понятными названиями для признаков
rename_dict = {
    'salary_in_usd': 'Зарплата (USD)',
    'experience_level': 'Уровень опыта',
    'employment_type': 'Тип трудоустройства', 
    'remote_ratio': 'Удаленная работа',
    'company_size': 'Размер компании'
}
# Переименовываем в таблице
df_ds_encoded = df_ds_encoded.rename(columns=rename_dict)

# Рассчитаем корреляцию по Спирману (так как признаки категориальные кроме з/п)
correlation_matrix = df_ds_encoded.corr(method='spearman')

# Строим тепловую карту корреляций
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, 
            annot=True,
            fmt='.2f',
            cmap='RdBu_r')

plt.title('Матрица корреляций между признаками', fontweight='bold')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()

# Для отображения раскомментировать plt.show()
# plt.show()
plt.close()

<div align="center">
<image src="data/image/plt_12.png"></div>

**Вывод**. Для Data Scientist наблюдается усиление влияния опыта на зарплату (корреляция 0.61 против 0.55 по всем должностям). Это указывает на то, что для DS-позиций уровень квалификации является ещё более значимым фактором оплаты труда. Как и в общем случае, остальные факторы демонстрируют слабые связи, подтверждая универсальность выявленной закономерности.

Подведем итоги. Выведем основные вопросы/утверждения касающиеся нашего исследования, которые проверим далее статистическими тестами:
1. Растет ли ежегодно зарплата у Data Science специалистов;
2. Data Scientist получают больше Data Engineer начиная с 2022 года;
3. Уровень опыта оказывает существенное влияние на заработную плату;
4. Полностью удаленные работники получают больше остальных;
5. Зарплаты в США выше, чем в других странах;
6. Размер компании влияет на уровень зарплат у Data Scientist;
7. Имеется прямая зависимость между размером компании и наличием должностей Data Scientist и Data Engineer.

### **2.2. Статистический анализ данных**

#### **Подготовка**

Для начала зададим некоторые функции: функция проверки на нормальность, проверки равенства дисперсий, и отклонении или принятии нулевой гипотезы, функция помогающая выбрать необходимый тест. Внутри функции пропишем уровень значимости $\alpha = 0,05$.

In [23]:
# Определяю необходимые мне функции
def get_norm(data, alpha=0.05):
    """Функция, помогающая определить нормально ли распределение.
    
    Args:
        data (array-like): данные для проверки на нормальность
        alpha (float): уровень значимости, по умолчанию 0.05
        
    Returns:
        str: подсказка о нормальности распределения
    """
    _, p_value = stats.shapiro(data)
    print(f'p-value = {p_value:.3f}')
    
    if p_value <= alpha:
        print(f'p-значение меньше, чем заданный уровень значимости {alpha}. '
              'Распределение отлично от нормального.')
    else:
        print(f'p-значение больше, чем заданный уровень значимости {alpha}. '
              'Распределение является нормальным.')


def get_var(data_1, data_2, alpha=0.05):
    """Функция, помогающая определить одинаковые ли дисперсии.
    
    Args:
        data_1 (array-like): первая выборка для сравнения
        data_2 (array-like): вторая выборка для сравнения
        alpha (float): уровень значимости, по умолчанию 0.05
        
    Returns:
        str: подсказка о равенстве дисперсий
    """
    _, p_value = stats.levene(data_1, data_2)
    print(f'p-value = {p_value:.3f}')
    
    if p_value <= alpha:
        print('Дисперсии не одинаковы')
    else:
        print('Дисперсии одинаковы')


def get_gip(p_value, alpha=0.05):
    """Функция, помогающая принять или отвергнуть нулевую гипотезу.
    
    Args:
        p_value (float): p-value статистического теста
        alpha (float): уровень значимости, по умолчанию 0.05
        
    Returns:
        str: подсказка о результате проверки гипотезы
    """
    print(f'p-value = {p_value:.3f}')
    
    if p_value <= alpha:
        print(f'p-значение меньше, чем заданный уровень значимости {alpha}. '
              'Отвергаем нулевую гипотезу.')
    else:
        print(f'p-значение больше, чем заданный уровень значимости {alpha}. '
              'У нас нет оснований отвергнуть нулевую гипотезу.')

def recommend_test(n_groups, is_dependent, is_normal):
    """Рекомендует статистический тест на основе характеристик данных.
    
    Args:
        n_groups (int): количество групп для сравнения (1, 2, 3+)
        is_dependent (int): 1 если группы зависимые, 0 если независимые
        is_normal (int): 1 если распределение нормальное, 0 если нет
        
    Returns:
        str: рекомендация по статистическому тесту
    """
    # Проверка валидности входных данных
    if is_dependent not in [0, 1] or is_normal not in [0, 1]:
        return "Ошибка: некорректные входные параметры"
    
    # Логика выбора теста
    if n_groups == 1:
        if is_normal:
            test = "Одновыборочный t-критерий: scipy.stats.ttest_1samp"
        else:
            test = ("Критерий знаков (для одной выборки): "
                    "statsmodels.stats.descriptivestats.sign_test")
        
    elif n_groups == 2:
        if is_dependent:
            if is_normal:
                test = "Парный t-критерий: scipy.stats.ttest_rel"
            else:
                test = "Критерий Уилкоксона: scipy.stats.wilcoxon"
        else:
            if is_normal:
                test = ("Двухвыборочный t-критерий: scipy.stats.ttest_ind. "
                        "Надо проверить равенство дисперсий")
            else:
                test = "U-критерий Манна-Уитни: scipy.stats.mannwhitneyu"
            
    else:  # n_groups >= 3
        if is_dependent:
            if is_normal:
                test = "ANOVA с повторными измерениями: statsmodels.stats.anova.AnovaRM"
            else:
                test = "Критерий Фридмана: scipy.stats.friedmanchisquare"
        else:
            if is_normal:
                test = ("ANOVA: scipy.stats.f_oneway. "
                        "Проверь дисперсию, если не равна используй is_normal=0")
            else:
                test = "Критерий Краскела-Уоллиса: scipy.stats.kruskal"
    
    return f"Рекомендуемый тест: {test}"

Таблица напоминание:

|**Нулевая гипотеза**|**Альтернативная гипотеза**|<center> **Тип**|
|--|--|--|
|$H_0 : μ_1 = μ_2$ |$H_1: \mu_1 \ne \mu_2$<br>|двухсторонняя, `alternative='two-sided'`|
|$H_0: \mu_1 \geq \mu_2$|$H_1: \mu_1 < \mu_2$|левосторонняя, `alternative='less'`|
|$H_0: \mu_1 \leq \mu_2$|$H_1: \mu_1 > \mu_2$|правосторонняя, `alternative='greater'`|

#### **Проверка гипотез**

**1. Растет ли ежегодно зарплата у Data Science специалистов.**

Проверим сначала меняется ли в целом зарплата у DS специалистов с каждым годом:

- Нулевая гипотеза: медианная заработная плата в 2020, 2021 и 2022 году одинаковая (у специалистов DS).

     $H_0 : \mu_1 = \mu_2 = \mu_3$

- Альтернативная гипотеза: медианная заработная плата в 2020, 2021 и 2022 году менялась (у специалистов DS).

     $H_0 : \mu_1 \ne \mu_2 \ne \mu_3$

Проверим данные на нормальность распределения:

In [24]:
# ds_mask - маска DS вакансий задана ранее
salary_ds_20 = data[ds_mask & (data['work_year']==2020)]['salary_in_usd']
salary_ds_21 = data[ds_mask & (data['work_year']==2021)]['salary_in_usd']
salary_ds_22 = data[ds_mask & (data['work_year']==2022)]['salary_in_usd']

get_norm(salary_ds_20)
get_norm(salary_ds_21)
get_norm(salary_ds_22)

p-value = 0.000
p-значение меньше, чем заданный уровень значимости 0.05. Распределение отлично от нормального.
p-value = 0.107
p-значение больше, чем заданный уровень значимости 0.05. Распределение является нормальным.
p-value = 0.676
p-значение больше, чем заданный уровень значимости 0.05. Распределение является нормальным.


In [25]:
# Выберем подходящий тест
recommend_test(3,0,0)

'Рекомендуемый тест: Критерий Краскела-Уоллиса: scipy.stats.kruskal'

In [26]:
_, p = stats.kruskal(salary_ds_20, salary_ds_21, salary_ds_22)
get_gip(p)

p-value = 0.000
p-значение меньше, чем заданный уровень значимости 0.05. Отвергаем нулевую гипотезу.


**Вывод:** таким образом мы можем подтвердить, что уровень заработной платы сотрудников DS изменяется с каждым годом. 
Но данная гипотеза не подтверждает рост, подтверждает лишь изменчивость, проверим дополнительно именно рост между 2020 и 2021, 2021 и 2022 годами.

Сформулируем гипотезы в математическом виде:

- Нулевые гипотезы:.
  1. Медианная заработная плата в 2020 >= 2021 (у специалистов DS).

     $H_0 : \mu_1 >= \mu_2$
  
  2. Медианная заработная плата в 2021 >= 2022 году (у специалистов DS).

     $H_0 : \mu_2 >= \mu_3$

- Альтернативные гипотезы: 
  1. Медианная заработная плата в 2020 < 2021 (у специалистов DS).

     $H_1 : \mu_1 < \mu_2$
  
  2. Медианная заработная плата в 2021 < 2022 (у специалистов DS).
  
     $H_1 : \mu_2 < \mu_3$

Альтернативные гипотезы левостороннии, `alternative='less'

Выберем тест:

In [27]:
# Выберем подходящий тест для первой гипотезы
print(f'Для первой гипотезы {recommend_test(2,0,0)}')
# Выберем подходящий тест для второй гипотезы
print(f'Для второй гипотезы {recommend_test(2,0,1)}') # значения 21 и 22 года распределены нормально

Для первой гипотезы Рекомендуемый тест: U-критерий Манна-Уитни: scipy.stats.mannwhitneyu
Для второй гипотезы Рекомендуемый тест: Двухвыборочный t-критерий: scipy.stats.ttest_ind. Надо проверить равенство дисперсий


In [28]:
# Проверка равенство дисперсий для 21 и 22 года
get_var(salary_ds_21, salary_ds_22)

p-value = 0.264
Дисперсии одинаковы


In [29]:
_, p_1 = stats.mannwhitneyu(salary_ds_20, salary_ds_21, alternative='less')
get_gip(p_1)
_, p_2 = stats.ttest_ind(salary_ds_21, salary_ds_22, alternative='less', equal_var=True)
get_gip(p_2)

p-value = 0.705
p-значение больше, чем заданный уровень значимости 0.05. У нас нет оснований отвергнуть нулевую гипотезу.
p-value = 0.000
p-значение меньше, чем заданный уровень значимости 0.05. Отвергаем нулевую гипотезу.


**Вывод:** Таким образом, тесты подтвердили ранее полученные результаты, между 2020-2021 годом рост зарплат отсутствовал, в 2022 году зарплата выросла.

**2. Data Scientist получают больше Data Engineer начиная с 2022 года**

- Нулевая гипотеза: медианная заработная плата в 2022 году у специалистов DS <= DE.

     $H_0 : \mu_1 <= \mu_2$

- Альтернативная гипотеза: медианная заработная плата в 2022 году у специалистов DS > DE.

     $H_0 : \mu_1 > \mu_2$

Альтернативная гипотеза правосторонняя, `alternative='greater'`
Проверим данные на нормальность распределения:

In [30]:
# ds_mask, de_mask - маски заданы ранее
salary_ds_22 = data[ds_mask & (data['work_year']==2022)]['salary_in_usd']
salary_de_22 = data[de_mask & (data['work_year']==2022)]['salary_in_usd']

get_norm(salary_ds_20)
get_norm(salary_de_22)

p-value = 0.000
p-значение меньше, чем заданный уровень значимости 0.05. Распределение отлично от нормального.
p-value = 0.007
p-значение меньше, чем заданный уровень значимости 0.05. Распределение отлично от нормального.


In [31]:
# Выберем подходящий тест для гипотезы
recommend_test(2,0,0)

'Рекомендуемый тест: U-критерий Манна-Уитни: scipy.stats.mannwhitneyu'

In [32]:
_, p = stats.mannwhitneyu(salary_ds_22, salary_de_22, alternative='greater')
get_gip(p)

p-value = 0.160
p-значение больше, чем заданный уровень значимости 0.05. У нас нет оснований отвергнуть нулевую гипотезу.


**Вывод:** Мы не можем отвергнуть нулевую гипотезу. Хотя медианная зарплата Data Scientist в 2022 году выше, чем у Data Engineer, статистически значимых различий на уровне α=0.05 не обнаружено.

**3. Уровень опыта оказывает существенное влияние на заработную плату**

- Нулевая гипотеза: медианная заработная плата у специалистов разного опыта работы (EN, MI, SE, EX) одинакова.

     $H_0 : \mu_1 = \mu_2 = \mu_3 = \mu_4$

- Альтернативная гипотеза: медианная заработная плата у специалистов разного опыта работы (EN, MI, SE, EX) не одинакова.

     $H_0 : \mu_1 \ne \mu_2 \ne \mu_3 \ne \mu_4$

Проверим данные на нормальность распределения:

In [33]:
# Маски
entry_mask = data['experience_level'] == 'EN'
mid_mask = data['experience_level'] == 'MI'
senior_mask = data['experience_level'] == 'SE'
executive_mask = data['experience_level'] == 'EX'

# Необходимые таблицы
salary_en = data[entry_mask]['salary_in_usd']
salary_mi = data[mid_mask]['salary_in_usd']
salary_se = data[senior_mask]['salary_in_usd']
salary_ex = data[executive_mask]['salary_in_usd']

# Проверка на нормальность
get_norm(salary_en)
get_norm(salary_mi)
get_norm(salary_se)
get_norm(salary_ex)

p-value = 0.000
p-значение меньше, чем заданный уровень значимости 0.05. Распределение отлично от нормального.
p-value = 0.000
p-значение меньше, чем заданный уровень значимости 0.05. Распределение отлично от нормального.
p-value = 0.000
p-значение меньше, чем заданный уровень значимости 0.05. Распределение отлично от нормального.
p-value = 0.001
p-значение меньше, чем заданный уровень значимости 0.05. Распределение отлично от нормального.


In [34]:
# Выберем подходящий тест для гипотезы
recommend_test(4,0,0)

'Рекомендуемый тест: Критерий Краскела-Уоллиса: scipy.stats.kruskal'

In [35]:
_, p = stats.kruskal(salary_en, salary_mi, salary_se, salary_ex)
get_gip(p)

p-value = 0.000
p-значение меньше, чем заданный уровень значимости 0.05. Отвергаем нулевую гипотезу.


Таким образом мы можем подтвердить, что уровень заработной платы сотрудников изменяется с изменением опыта. Однако данная гипотеза не подтверждает рост, подтверждает лишь изменчивость, проверим дополнительно именно рост заработной платы с повышением каждого уровня опыта.

Сформулируем гипотезы в математическом виде:

- Нулевые гипотезы:
    1. медианная заработная плата EN >= MI
      
       $H_0 : \mu_1 >= \mu_2$

    2. медианная заработная плата MI >= SE
      
       $H_0 : \mu_2 >= \mu_3$

    3. медианная заработная плата SE >= EX
      
       $H_0 : \mu_3 >= \mu_4$

- Альтернативные гипотезы:

    1. медианная заработная плата EN < MI
      
       $H_1 : \mu_1 < \mu_2$

    2. медианная заработная плата MI < SE
      
       $H_1 : \mu_2 < \mu_3$

    3. медианная заработная плата SE < EX
      
       $H_1 : \mu_3 < \mu_4$

Альтернативные гипотезы левосторонние, `alternative='less'

Выберем тест:

In [36]:
# Выберем подходящий тест для гипотез (будет одинаковый у всех)
recommend_test(2,0,0)

'Рекомендуемый тест: U-критерий Манна-Уитни: scipy.stats.mannwhitneyu'

In [37]:
# Проверяем гипотезы
_, p_1 = stats.mannwhitneyu(salary_en, salary_mi, alternative='less')
_, p_2 = stats.mannwhitneyu(salary_mi, salary_se, alternative='less')
_, p_3 = stats.mannwhitneyu(salary_se, salary_ex, alternative='less')
get_gip(p_1)
get_gip(p_2)
get_gip(p_3)

p-value = 0.000
p-значение меньше, чем заданный уровень значимости 0.05. Отвергаем нулевую гипотезу.
p-value = 0.000
p-значение меньше, чем заданный уровень значимости 0.05. Отвергаем нулевую гипотезу.
p-value = 0.002
p-значение меньше, чем заданный уровень значимости 0.05. Отвергаем нулевую гипотезу.


**Вывод:** Статистический анализ подтвердил, что уровень опыта является значимым фактором, определяющим заработную плату, с наблюдаемым ростом доходов от начинающих специалистов (EN) к руководящим позициям (EX).

**4. Работники на полной удаленке получают больше остальных**

Попробуем ответить на 2 вопроса: больше ли зарплата работников на полной удаленке, чем у совмещающих сотрудников (remote_ratio = 50)? больше ли зарплата работников на полной удаленке (remote_ratio = 100), чем у офисных работников (remote_ratio = 0)?


- Нулевая гипотеза:

  1. медианная заработная плата у специалистов работающих на удаленке меньше или равна чем у совмещающих офис и удаленку

     $H_0 : \mu_1 <= \mu_2$

  2. медианная заработная плата у специалистов работающих на удаленке меньше или равна чем у работающих в офисе
    
     $H_0 : \mu_1 <= \mu_3$

- Альтернативная гипотеза:

  1. медианная заработная плата у специалистов работающих на удаленке больше чем у совмещающих офис и удаленку
  
     $H_1 : \mu_1 > \mu_2$

  2. медианная заработная плата у специалистов работающих на удаленке больше чем у работающих в офисе
  
     $H_1 : \mu_1 > \mu_3$

Альтернативные гипотезы правостороннии, `alternative='greater'`

Проведем тест:

In [38]:
# Маски
mask_remote_100 = data['remote_ratio'] == 100
mask_remote_50 = data['remote_ratio'] == 50
mask_remote_0 = data['remote_ratio'] == 0

# Необходимые таблицы
salary_remote_100 = data[mask_remote_100]['salary_in_usd']
salary_remote_50 = data[mask_remote_50]['salary_in_usd']
salary_remote_0 = data[mask_remote_0]['salary_in_usd']

# Проверка на нормальность
get_norm(salary_remote_100)
get_norm(salary_remote_50)
get_norm(salary_remote_0)

p-value = 0.000
p-значение меньше, чем заданный уровень значимости 0.05. Распределение отлично от нормального.
p-value = 0.000
p-значение меньше, чем заданный уровень значимости 0.05. Распределение отлично от нормального.
p-value = 0.000
p-значение меньше, чем заданный уровень значимости 0.05. Распределение отлично от нормального.


In [39]:
# Выберем подходящий тест для гипотез (будет одинаковый у всех)
recommend_test(2,0,0)

'Рекомендуемый тест: U-критерий Манна-Уитни: scipy.stats.mannwhitneyu'

In [40]:
# Проверяем гипотезы
_, p_1 = stats.mannwhitneyu(salary_remote_100, salary_remote_50, alternative='greater')
_, p_2 = stats.mannwhitneyu(salary_remote_100, salary_remote_0, alternative='greater')
get_gip(p_1)
get_gip(p_2)

p-value = 0.000
p-значение меньше, чем заданный уровень значимости 0.05. Отвергаем нулевую гипотезу.
p-value = 0.018
p-значение меньше, чем заданный уровень значимости 0.05. Отвергаем нулевую гипотезу.


**Вывод:** Статистический анализ показал, что сотрудники на полной удаленке (remote_ratio=100) получают значимо больше как по сравнению с сотрудниками на гибридном формате (remote_ratio=50), так и по сравнению с офисными сотрудниками (remote_ratio=0).

Для полной картины можем сравнить гибридный тип работы и офисный. Согласно графикам гибридный тип работы самый низкооплачиваемый. Проверим это:

- Нулевая гипотеза: медианная заработная плата у специалистов совмещающих удаленку и офис выше или равна чем у работающих в офисе.

  $H_0 : \mu_2 >= \mu_3$

- Альтернативная гипотеза: медианная заработная плата у специалистов совмещающих удаленку и офис ниже чем у работающих в офисе.

  $H_1 : \mu_2 < \mu_3$

Альтернативная гипотеза левосторонняя, `alternative='less', данные не распределены нормально, не зависимы. Тест тот же самый, что и для предыдущих сравнений.

In [41]:
# Проверяем гипотезу
_, p = stats.mannwhitneyu(salary_remote_50, salary_remote_0, alternative='less')
get_gip(p)

p-value = 0.001
p-значение меньше, чем заданный уровень значимости 0.05. Отвергаем нулевую гипотезу.


**Вывод:** Альтернативная гипотеза подтвердилась. 
Таким образом, анализ выявил четкую иерархию заработных плат в зависимости от формата работы: сотрудники на полной удаленке получают наибольший доход, затем следуют офисные работники, а гибридный формат ассоциируется с наименьшими заработными платами.

**5. Зарплаты в США выше, чем в других странах**

- Нулевая гипотеза: медианная заработная плата у специалистов работающих в США ниже или равна всем остальным странам.

  $H_0 : \mu_1 <= \mu_2$

- Альтернативная гипотеза: медианная заработная плата у специалистов работающих в США выше всех остальных стран.

  $H_1 : \mu_1 > \mu_2$

Альтернативная гипотеза правосторонняя, `alternative='greater'

Проведем тесты:

In [42]:
# Объединяем все US и страны кроме US
mask_us = data['company_location'] == 'US'
mask_other = data['company_location'] != 'US'

salary_us = data[mask_us]['salary_in_usd']
salary_other = data[mask_other]['salary_in_usd']

# Проверка на нормальность
get_norm(salary_us)
get_norm(salary_other)

p-value = 0.000
p-значение меньше, чем заданный уровень значимости 0.05. Распределение отлично от нормального.
p-value = 0.000
p-значение меньше, чем заданный уровень значимости 0.05. Распределение отлично от нормального.


In [43]:
# Выберем подходящий тест для гипотезы
recommend_test(2,0,0)

'Рекомендуемый тест: U-критерий Манна-Уитни: scipy.stats.mannwhitneyu'

In [44]:
# Проверяем гипотезу
_, p = stats.mannwhitneyu(salary_us, salary_other, alternative='greater')
get_gip(p)

p-value = 0.000
p-значение меньше, чем заданный уровень значимости 0.05. Отвергаем нулевую гипотезу.


**Вывод:** уровень заработной платы в США для всех вакансий выше чем заработная плата в других странах. 

Проверим теперь данное утверждение для вакансий в сфере Data Science:

- Нулевая гипотеза: медианная заработная плата у специалистов DS работающих в США ниже или равна всем остальным странам.

  $H_0 : \mu_1 <= \mu_2$

- Альтернативная гипотеза: медианная заработная плата у специалистов DS работающих в США выше всех остальных стран.

  $H_1 : \mu_1 > \mu_2$

Альтернативная гипотеза правосторонняя, `alternative='greater'`

Проведем тесты:

In [45]:
# Маски заданы ранее
salary_us_ds = data[mask_us & ds_mask]['salary_in_usd']
salary_other_ds = data[mask_other & ds_mask]['salary_in_usd']

# Проверка на нормальность
get_norm(salary_us_ds)
get_norm(salary_other_ds)

p-value = 0.000
p-значение меньше, чем заданный уровень значимости 0.05. Распределение отлично от нормального.
p-value = 0.008
p-значение меньше, чем заданный уровень значимости 0.05. Распределение отлично от нормального.


In [46]:
# Тест тот же самый, параметры не изменились
# Проверяем гипотезу
_, p = stats.mannwhitneyu(salary_us_ds, salary_other_ds, alternative='greater')
get_gip(p)

p-value = 0.000
p-значение меньше, чем заданный уровень значимости 0.05. Отвергаем нулевую гипотезу.


**Выводы:** Заработная плата сотрудников, работающих в США значительно больше, чем в других странах, как для всей выборки, так и для профессии - Data Science

**6. Размер компании влияет на уровень зарплат у Data Scientist**

- Нулевая гипотеза: медианная заработная плата у специалистов DS работающих в малых, средних и больших компаниях одинаков.

  $H_0 : \mu_1 = \mu_2 = \mu_3$

- Альтернативная гипотеза: медианная заработная плата у специалистов DS работающих в малых, средних и больших компаниях различается.

  $H_0 : \mu_1 \ne \mu_2 \ne \mu_3$

Альтернативная гипотеза двухсторонняя

Проведем тесты:

In [47]:
# Зададим маски
mask_s_size = data['company_size'] == 'S'
mask_m_size = data['company_size'] == 'M'
mask_l_size = data['company_size'] == 'L'

# Необходимые таблицы
salary_s_size_ds = data[mask_s_size & ds_mask]['salary_in_usd']
salary_m_size_ds = data[mask_m_size & ds_mask]['salary_in_usd']
salary_l_size_ds = data[mask_l_size & ds_mask]['salary_in_usd']

# Проверка на нормальность
get_norm(salary_s_size_ds)
get_norm(salary_m_size_ds)
get_norm(salary_l_size_ds)

p-value = 0.189
p-значение больше, чем заданный уровень значимости 0.05. Распределение является нормальным.
p-value = 0.897
p-значение больше, чем заданный уровень значимости 0.05. Распределение является нормальным.
p-value = 0.000
p-значение меньше, чем заданный уровень значимости 0.05. Распределение отлично от нормального.


In [48]:
# Выберем подходящий тест для гипотезы
recommend_test(3,0,0)

'Рекомендуемый тест: Критерий Краскела-Уоллиса: scipy.stats.kruskal'

In [49]:
_, p = stats.kruskal(salary_s_size_ds, salary_m_size_ds, salary_l_size_ds)
get_gip(p)

p-value = 0.000
p-значение меньше, чем заданный уровень значимости 0.05. Отвергаем нулевую гипотезу.


**Вывод:** Таким образом можно дать однозначный ответ, что размер компании влияет на заработную плату.

Согласно проведенным ранее исследованиям, самый большой доход наблюдается в компаниях среднего размера. Можно дополнительно убедится в этом. Проверим для этого 2 гипотезы:
- Больше ли заработная плата в средних компаниях чем в малых
- Больше ли заработная плата в средних компаниях чем в крупных

Сформулируем нулевые и альтернативные гипотезы:
- Нулевые гипотезы: 
  1. медианная заработная плата в средних компаниях меньше или равна чем в малых.

     $H_0 : \mu_2 <= \mu_1$
  
  2. медианная заработная плата в средних компаниях меньше или равна чем в крупных.

     $H_0 : \mu_2 <= \mu_3$

- Альтернативные гипотезы: 
  1. медианная заработная плата в средних компаниях больше чем в малых.

     $H_1 : \mu_2 > \mu_1$
  2. медианная заработная плата в средних компаниях больше чем в крупных.

     $H_1 : \mu_2 > \mu_3$

Альтернативные гипотезы правосторонние, `alternative='greater'`

In [50]:
# Выберем подходящий тест для 1 гипотезы,
# данные в которой имеют нормальное распределение
print(recommend_test(2,0,1))
# Выберем подходящий тест для 2 гипотезы
print(recommend_test(2,0,0))

Рекомендуемый тест: Двухвыборочный t-критерий: scipy.stats.ttest_ind. Надо проверить равенство дисперсий
Рекомендуемый тест: U-критерий Манна-Уитни: scipy.stats.mannwhitneyu


In [51]:
# Проверяем дисперсию, для 1 гипотезы
get_var(salary_s_size_ds, salary_m_size_ds)

p-value = 0.018
Дисперсии не одинаковы


In [52]:
# Проверяем гипотезы
_, p_1 = stats.ttest_ind(salary_m_size_ds, salary_s_size_ds, alternative='greater', equal_var=False)
_, p_2 = stats.mannwhitneyu(salary_m_size_ds, salary_l_size_ds, alternative='greater')
get_gip(p_1)
get_gip(p_2)

p-value = 0.000
p-значение меньше, чем заданный уровень значимости 0.05. Отвергаем нулевую гипотезу.
p-value = 0.024
p-значение меньше, чем заданный уровень значимости 0.05. Отвергаем нулевую гипотезу.


**Вывод:** Статистические тесты подтверждают наблюдения сделанные ранее на графике. Размер компании оказывает влияние на заработную плату DS специалистов. Самый большой доход у компаний средних размеров.

**6. Имеется прямая зависимость между размером компании и наличием должностей Data Scientist и Data Engineer**

- Нулевая гипотеза: размер компании и наличие должностей DS/DE не связаны.
- Альтернативная гипотеза: размер компании и наличие должностей DS/DE связаны.

Для проверки этой гипотезы нужно использовать тесты на связь категориальных переменных (критерий хи-квадрат).

Проведем тесты:

In [53]:
# Создаем таблицу сопряженности
table_ds_de = pd.crosstab(
    data['company_size'],
    data['job_title'].isin(['Data Scientist', 'Data Engineer'])
)

print("Таблица сопряженности:")
display(table_ds_de)

# Проводим тест
_, p, _, _ = stats.chi2_contingency(table_ds_de)
get_gip(p)

Таблица сопряженности:


job_title,False,True
company_size,,
L,116,77
M,143,147
S,55,27


p-value = 0.005
p-значение меньше, чем заданный уровень значимости 0.05. Отвергаем нулевую гипотезу.


**Вывод:** Тест подтвердил, что имеется прямая взаимосвязь между размером компании и наличием должностей DS, DE.

## 3. Выводы

Подводя итоги, в качестве вывода сформулируем бизнес вопросы и полученные в исследовании ответы на них
|**Бизнес-вопрос**|**Ответ**|
|--|--|
|Наблюдается ли ежегодный рост зарплат у специалистов Data Scientist?|Статистически значимый рост зарплат наблюдается в 2022 году относительно предыдущих периодов. Динамика между 2020 и 2021 годами не показала значимых изменений.|
|Как соотносятся зарплаты Data Scientist и Data Engineer в 2022 году?|У нас нет оснований для утверждения, что заработная плата DS выше чем у DE, хотя медианная зарплата DS в 2022 году выше, чем у DE, статистически значимых различий не обнаружено.|
|Как соотносятся зарплаты специалистов Data Scientist в компаниях различных размеров?|Наибольшие зарплаты наблюдаются в средних компаниях, затем следуют крупные, а малые компании предлагают наименьшее вознаграждение.|
|Есть ли связь между наличием должностей Data Scientist и Data Engineer и размером компании?|Исследование подтвердило, что имеется прямая взаимосвязь между размером компании и наличием должностей DS, DE.|
|Оказывает ли влияние уровень опыта на заработную плату?|Статистический анализ подтвердил, что уровень опыта является значимым фактором, определяющим заработную плату, с ростом доходов от начинающих специалистов (EN) к руководящим позициям (EX).|
|Оказывает ли влияние на заработную плату тип работы (офис, удаленка, гибрид)?|Полная удаленка ассоциирована с наибольшими зарплатами, затем следует офисный формат, гибридный показывает наименьшие значения.|
|Зарплаты в США выше, чем в других странах?|Заработная плата сотрудников, работающих в США больше, чем в других странах, как для всей выборки, так и для профессии - Data Science|